In [7]:
import requests
import json
import matplotlib
import re
import pydot
from graphviz import Digraph
from graphviz import Source
from utils import Utils
import networkx as nx

In [8]:
import xml.etree.ElementTree as ET
tree = ET.parse('train-data.xml')

In [9]:
from collections import defaultdict
from networkx.readwrite import json_graph

In [10]:
instances = tree.findall('instance')

In [11]:
texts = {}
id_to_questions = defaultdict(list)
id_to_answers = defaultdict(list)
id_to_correct = defaultdict(list)
id_to_type = defaultdict(list)

First I process the data, read it into the above defined dicts

In [12]:
count = 0
for inst in instances:
    text = inst.find('text').text
    texts[count] = text
    questions = inst.find('questions').findall('question')
    for q in questions:
        id_to_questions[count].append(q.attrib['text'])
        id_to_type[count].append(q.attrib['type'])
        ans = q.findall('answer')
        ans_text = []
        corr_text = []
        for a in ans:
            ans_text.append(a.attrib['text'])
            corr_text.append(a.attrib['correct'])
        id_to_answers[count].append(ans_text)
        id_to_correct[count].append(corr_text)
    count+=1
        

Now first I process the texts, so it doesnt need processing for every question.

I defined two new endpoint:
* rallyexp - In this endpoint, I process the question with an answer and construct a new graph from the two of them. It uses the definitions of the words.
* rallyabs - Same as the rallyexp, only with the abstraction rules

In [ ]:
utils = Utils()
text_edges = {}
for j in texts.keys():
    print(j)
    data = {'sentence':   texts[j]}
    data_json = json.dumps(data)
    payload = {'json_payload': data_json}
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    try:
        r = requests.post("http://hlt.bme.hu/4lang/senexp", data=data_json, headers=headers)
        s_machines = r.json()['sentence']
    except ValueError:
        continue
    g_sen = json_graph.adjacency.adjacency_graph(s_machines)

    edges_text = utils.get_edges(g_sen)
    text_edges[j] = edges_text

First I retrieve the text from the dict, after that I build two graphs with the two question-answer pairs and then calculate the
edge similarities to the text's graphs. The more similar graph will be the True answer and the other will be False

In [19]:
question1 = {}
question2 = {}
def get_pred(text, question, answers):   
    edges_text = text_edges[text]
    
    data = {'question':   question, 'answer': answers[0]}
    data_json = json.dumps(data)
    payload = {'json_payload': data_json}
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    r = requests.post("http://hlt.bme.hu/4lang/rallyexp", data=data_json, headers=headers)
    first_machines = r.json()['rally']
    g_que1 = json_graph.adjacency.adjacency_graph(first_machines)
    
    edges_question1 = utils.get_edges(g_que1)
    question1[text] = g_que1
    
    data = {'question':   question, 'answer': answers[1]}
    data_json = json.dumps(data)
    payload = {'json_payload': data_json}
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    r = requests.post("http://hlt.bme.hu/4lang/rallyexp", data=data_json, headers=headers)
    second_machines = r.json()['rally']
    g_que2 = json_graph.adjacency.adjacency_graph(second_machines)
        
    question2[text] = g_que2
    edges_question2 = utils.get_edges(g_que2)
    
    sim_to_first = utils.asim_jac(edges_text, edges_question1)
    sim_to_second = utils.asim_jac(edges_text, edges_question2)
    return ['True', 'False'] if sim_to_first >= sim_to_second else ['False', 'True']

Here I iterate over the dataset, and run the above defined method only with questions that are not commonsense, and only if there are a question word in the question. I close out one worded answers as well.

In [ ]:
right = 0
all_q = 0 
qst_words = ["why", "where", "how", "when", "what", "who"]
correct = []
pred = []
for i in texts.keys():
    for quest, typ, corr, ans in zip(id_to_questions[i], id_to_type[i], id_to_correct[i], id_to_answers[i]):
        all_q += 1
        right_q = [i for i in qst_words if i in quest.lower()]
        if len(right_q) > 0 and typ == 'text':
            try:
                prediction = get_pred(i, quest, ans)
            except:
                continue
            correct.append(int(corr[0] == 'True'))
            correct.append(int(corr[1] == 'True'))
            pred.append(int(prediction[0] == 'True'))
            pred.append(int(prediction[1] == 'True'))
            print(right)
            right+=1
print(right)
print(all_q)

In [22]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [23]:
accuracy = accuracy_score(correct, pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(correct, pred, average='binary')

In [31]:
print("accuracy: " + str(accuracy))

accuracy: 0.6837200456447319


The rest is the same as above, only using the abstraction endpoint with expanded text graphs.

In [ ]:
text_edges_abstract = {}
text_abstract = {}
for j in texts.keys():
    print(j)        
    data = {'sentence':   texts[j]}
    data_json = json.dumps(data)
    payload = {'json_payload': data_json}
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    try:
        r = requests.post("http://hlt.bme.hu/4lang/senabs", data=data_json, headers=headers)
        s_machines = r.json()['sentence']
    except ValueError:
        continue
    g_sen = json_graph.adjacency.adjacency_graph(s_machines)

    edges_text = utils.get_edges(g_sen)
    text_edges_abstract[j] = edges_text
    text_abstract[j] = g_sen

In [27]:
def get_pred_abs(text, question, answers):   
    edges_text = text_edges[text]
    
    data = {'question':   question, 'answer': answers[0]}
    data_json = json.dumps(data)
    payload = {'json_payload': data_json}
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    r = requests.post("http://hlt.bme.hu/4lang/rallyabs", data=data_json, headers=headers)
    first_machines = r.json()['rally']
    g_que1 = json_graph.adjacency.adjacency_graph(first_machines)
    
    edges_question1 = utils.get_edges(g_que1)
    question1[text] = g_que1
    
    data = {'question':   question, 'answer': answers[1]}
    data_json = json.dumps(data)
    payload = {'json_payload': data_json}
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    r = requests.post("http://hlt.bme.hu/4lang/rallyabs", data=data_json, headers=headers)
    second_machines = r.json()['rally']
    g_que2 = json_graph.adjacency.adjacency_graph(second_machines)
        
    question2[text] = g_que2
    edges_question2 = utils.get_edges(g_que2)
    
    sim_to_first = utils.asim_jac(edges_text, edges_question1)
    sim_to_second = utils.asim_jac(edges_text, edges_question2)
    return ['True', 'False'] if sim_to_first >= sim_to_second else ['False', 'True']

In [ ]:
right = 0
all_q = 0 
qst_words = ["why", "where", "how", "when", "what", "who"]
correct = []
pred = []
for i in texts.keys():
    for quest, typ, corr, ans in zip(id_to_questions[i], id_to_type[i], id_to_correct[i], id_to_answers[i]):
        all_q += 1
        right_q = [i for i in qst_words if i in quest.lower()]
        if len(right_q) > 0 and typ == 'text':
            try:
                prediction = get_pred_abs(i, quest, ans)
            except:
                continue
            correct.append(int(corr[0] == 'True'))
            correct.append(int(corr[1] == 'True'))
            pred.append(int(prediction[0] == 'True'))
            pred.append(int(prediction[1] == 'True'))
            print(right)
            right+=1
print(right)
print(all_q)

In [32]:
accuracy = accuracy_score(correct, pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(correct, pred, average='binary')

In [33]:
print("accuracy: " + str(accuracy))

accuracy: 0.5812095853936858


In [50]:
def get_pred_varied(text, question, answers, edge_type, ans_type):
    if edge_type == 0:
        edges_text = text_edges_abstract[text]
    elif edge_type == 1:
        edges_text = text_edges[text]
    
    if ans_type == 0:        
        data = {'question':   question, 'answer': answers[0]}
        data_json = json.dumps(data)
        payload = {'json_payload': data_json}
        headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
        r = requests.post("http://hlt.bme.hu/4lang/rallyabs", data=data_json, headers=headers)
        first_machines = r.json()['rally']
        g_que1 = json_graph.adjacency.adjacency_graph(first_machines)

        edges_question1 = utils.get_edges(g_que1)
        question1[text] = g_que1

        data = {'question':   question, 'answer': answers[1]}
        data_json = json.dumps(data)
        payload = {'json_payload': data_json}
        headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
        r = requests.post("http://hlt.bme.hu/4lang/rallyabs", data=data_json, headers=headers)
        second_machines = r.json()['rally']
        g_que2 = json_graph.adjacency.adjacency_graph(second_machines)

        question2[text] = g_que2
        edges_question2 = utils.get_edges(g_que2)
    elif ans_type == 1:
        data = {'question':   question, 'answer': answers[0]}
        data_json = json.dumps(data)
        payload = {'json_payload': data_json}
        headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
        r = requests.post("http://hlt.bme.hu/4lang/rallyexp", data=data_json, headers=headers)
        first_machines = r.json()['rally']
        g_que1 = json_graph.adjacency.adjacency_graph(first_machines)

        edges_question1 = utils.get_edges(g_que1)
        question1[text] = g_que1

        data = {'question':   question, 'answer': answers[1]}
        data_json = json.dumps(data)
        payload = {'json_payload': data_json}
        headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
        r = requests.post("http://hlt.bme.hu/4lang/rallyexp", data=data_json, headers=headers)
        second_machines = r.json()['rally']
        g_que2 = json_graph.adjacency.adjacency_graph(second_machines)

        question2[text] = g_que2
        edges_question2 = utils.get_edges(g_que2)
    
    sim_to_first = utils.asim_jac(edges_text, edges_question1)
    sim_to_second = utils.asim_jac(edges_text, edges_question2)
    return ['True', 'False'] if sim_to_first >= sim_to_second else ['False', 'True']

In [ ]:
right = 0
all_q = 0 
qst_words = ["why", "where", "how", "when", "what", "who"]
correct = []
pred = []
for i in texts.keys():
    for quest, typ, corr, ans in zip(id_to_questions[i], id_to_type[i], id_to_correct[i], id_to_answers[i]):
        all_q += 1
        right_q = [i for i in qst_words if i in quest.lower()]
        if len(right_q) > 0 and typ == 'text':
            try:
                prediction = get_pred_varied(i, quest, ans, 0, 0)
            except:
                continue
            correct.append(int(corr[0] == 'True'))
            correct.append(int(corr[1] == 'True'))
            pred.append(int(prediction[0] == 'True'))
            pred.append(int(prediction[1] == 'True'))
            print(right)
            right+=1
print(right)
print(all_q)

In [49]:
accuracy = accuracy_score(correct, pred)
accuracy

0.6610233973749287

In [ ]:
right = 0
all_q = 0 
qst_words = ["why", "where", "how", "when", "what", "who"]
correct = []
pred = []
for i in texts.keys():
    for quest, typ, corr, ans in zip(id_to_questions[i], id_to_type[i], id_to_correct[i], id_to_answers[i]):
        all_q += 1
        right_q = [i for i in qst_words if i in quest.lower()]
        if len(right_q) > 0 and typ == 'text':
            try:
                prediction = get_pred_varied(i, quest, ans, 0, 1)
            except:
                continue
            correct.append(int(corr[0] == 'True'))
            correct.append(int(corr[1] == 'True'))
            pred.append(int(prediction[0] == 'True'))
            pred.append(int(prediction[1] == 'True'))
            print(right)
            right+=1
print(right)
print(all_q)

In [55]:
accuracy = accuracy_score(correct, pred)
accuracy

0.5722708254089007